In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from catboost import CatBoost
import gc

In [2]:
def read_single_csv(input_path):
    df_chunk=pd.read_csv(input_path,chunksize=1000)
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
data_list = []
for date in range(8, 32):
    data_list.append(reduce_mem_usage(read_single_csv('./data/traffic/train_table_%02d_1205.gz'%date)))

In [ ]:
data_list.append(reduce_mem_usage(pd.read_csv('./data/test_table_1205.gz')))

In [ ]:
data_list = pd.concat(data_list)

In [ ]:
data_list = data_list.drop(columns='Unnamed: 0')

In [ ]:
data_list['topo_group'] = data_list['topo_group'].fillna(-1).astype(int)

In [ ]:
partition_group = pd.read_csv('./data/partition_group.csv')
data_list = pd.merge(left = data_list,right = partition_group,on='link',how='left')
data_list['sim_group'] = data_list['sim_group'].fillna(-1).astype(int)

In [3]:
category_features = ['topo_group','sim_group']

In [4]:
non_features = ['link','label','target_avg_week_5','target_avg_week_10','target_avg_week_15']

## 数据集划分

In [ ]:
train_offline = data_list[data_list['date'] <= 29]
val_offline = data_list[data_list['date'] == 29]

In [21]:
train_online = data_list[data_list['date'] <= 30]
test_online = data_list[data_list['date'] == 31]

In [23]:
data_list = None

In [25]:
gc.collect()

0

## Catboost 建模

In [26]:
para = {
    'task_type':'GPU',
    'loss_function' : 'MultiClass',
    'iterations' :15000,
    'learning_rate' : 0.03,
    'depth' : 8,
    'class_weights' : {1:0.2,2:0.2,3:0.6,4:0.6},
    'eval_metric': 'TotalF1',
    'early_stopping_rounds': 100,
    "max_ctr_complexity" : 2
}

In [ ]:
model = CatBoost(para)

model.fit(X = train_offline[[x for x in train_offline.columns if x not in non_features]],
        y = train_offline['label'],
        sample_weight = train_offline['date'].apply(lambda x: np.log(x + 2)),
        cat_features = category_features,
        eval_set = (val_offline[[x for x in train_offline.columns if x not in non_features]], val_offline['label']),
        verbose = True, plot=True)

val_predict = model.predict(val_offline[[x for x in train_offline.columns if x not in non_features]], prediction_type='Class')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6254921	test: 0.6669945	best: 0.6669945 (0)	total: 523ms	remaining: 2h 10m 38s
1:	learn: 0.7288028	test: 0.7567836	best: 0.7567836 (1)	total: 883ms	remaining: 1h 50m 20s
2:	learn: 0.7760787	test: 0.7926363	best: 0.7926363 (2)	total: 1.27s	remaining: 1h 45m 50s
3:	learn: 0.7890596	test: 0.8024760	best: 0.8024760 (3)	total: 1.63s	remaining: 1h 41m 53s
4:	learn: 0.7912095	test: 0.8040220	best: 0.8040220 (4)	total: 2.05s	remaining: 1h 42m 22s
5:	learn: 0.7924687	test: 0.8050907	best: 0.8050907 (5)	total: 2.48s	remaining: 1h 43m 15s
6:	learn: 0.7948433	test: 0.8068974	best: 0.8068974 (6)	total: 2.87s	remaining: 1h 42m 18s
7:	learn: 0.7963085	test: 0.8080050	best: 0.8080050 (7)	total: 3.27s	remaining: 1h 42m 15s
8:	learn: 0.7975174	test: 0.8090813	best: 0.8090813 (8)	total: 3.67s	remaining: 1h 41m 49s
9:	learn: 0.7977034	test: 0.8093799	best: 0.8093799 (9)	total: 4.09s	remaining: 1h 42m 18s
10:	learn: 0.7983708	test: 0.8099499	best: 0.8099499 (10)	total: 4.52s	remaining: 1h 42m 4

89:	learn: 0.8094128	test: 0.8198780	best: 0.8198780 (89)	total: 37.3s	remaining: 1h 42m 57s
90:	learn: 0.8095293	test: 0.8200568	best: 0.8200568 (90)	total: 37.7s	remaining: 1h 42m 57s
91:	learn: 0.8095911	test: 0.8200877	best: 0.8200877 (91)	total: 38.2s	remaining: 1h 43m 11s
92:	learn: 0.8096557	test: 0.8200940	best: 0.8200940 (92)	total: 38.7s	remaining: 1h 43m 17s
93:	learn: 0.8097412	test: 0.8201870	best: 0.8201870 (93)	total: 39.2s	remaining: 1h 43m 29s
94:	learn: 0.8098325	test: 0.8202271	best: 0.8202271 (94)	total: 39.6s	remaining: 1h 43m 26s
95:	learn: 0.8099563	test: 0.8203335	best: 0.8203335 (95)	total: 40s	remaining: 1h 43m 27s
96:	learn: 0.8100632	test: 0.8204671	best: 0.8204671 (96)	total: 40.4s	remaining: 1h 43m 23s
97:	learn: 0.8101528	test: 0.8206327	best: 0.8206327 (97)	total: 40.8s	remaining: 1h 43m 20s
98:	learn: 0.8102320	test: 0.8206704	best: 0.8206704 (98)	total: 41.2s	remaining: 1h 43m 20s
99:	learn: 0.8102910	test: 0.8208178	best: 0.8208178 (99)	total: 41.7s	r

176:	learn: 0.8152950	test: 0.8257815	best: 0.8257828 (175)	total: 1m 15s	remaining: 1h 45m 55s
177:	learn: 0.8153756	test: 0.8258950	best: 0.8258950 (177)	total: 1m 16s	remaining: 1h 45m 54s
178:	learn: 0.8154047	test: 0.8259756	best: 0.8259756 (178)	total: 1m 16s	remaining: 1h 45m 43s
179:	learn: 0.8154605	test: 0.8260005	best: 0.8260005 (179)	total: 1m 17s	remaining: 1h 45m 44s
180:	learn: 0.8155280	test: 0.8260820	best: 0.8260820 (180)	total: 1m 17s	remaining: 1h 45m 42s
181:	learn: 0.8155491	test: 0.8261009	best: 0.8261009 (181)	total: 1m 17s	remaining: 1h 45m 38s
182:	learn: 0.8155899	test: 0.8261492	best: 0.8261492 (182)	total: 1m 18s	remaining: 1h 45m 55s
183:	learn: 0.8156503	test: 0.8262114	best: 0.8262114 (183)	total: 1m 18s	remaining: 1h 45m 59s
184:	learn: 0.8157066	test: 0.8262779	best: 0.8262779 (184)	total: 1m 19s	remaining: 1h 46m
185:	learn: 0.8157367	test: 0.8262985	best: 0.8262985 (185)	total: 1m 19s	remaining: 1h 46m 5s
186:	learn: 0.8157787	test: 0.8263554	best: 0

262:	learn: 0.8184046	test: 0.8289357	best: 0.8289357 (262)	total: 1m 52s	remaining: 1h 45m 30s
263:	learn: 0.8184528	test: 0.8289625	best: 0.8289625 (263)	total: 1m 53s	remaining: 1h 45m 32s
264:	learn: 0.8184773	test: 0.8290134	best: 0.8290134 (264)	total: 1m 53s	remaining: 1h 45m 26s
265:	learn: 0.8185025	test: 0.8290461	best: 0.8290461 (265)	total: 1m 54s	remaining: 1h 45m 23s
266:	learn: 0.8185231	test: 0.8290791	best: 0.8290791 (266)	total: 1m 54s	remaining: 1h 45m 21s
267:	learn: 0.8185517	test: 0.8291052	best: 0.8291052 (267)	total: 1m 55s	remaining: 1h 45m 26s
268:	learn: 0.8185821	test: 0.8290844	best: 0.8291052 (267)	total: 1m 55s	remaining: 1h 45m 26s
269:	learn: 0.8185989	test: 0.8291004	best: 0.8291052 (267)	total: 1m 55s	remaining: 1h 45m 20s
270:	learn: 0.8186251	test: 0.8291395	best: 0.8291395 (270)	total: 1m 56s	remaining: 1h 45m 26s
271:	learn: 0.8186473	test: 0.8292347	best: 0.8292347 (271)	total: 1m 56s	remaining: 1h 45m 32s
272:	learn: 0.8186911	test: 0.8292658	be

...

12610:	learn: 0.8739217	test: 0.8884955	best: 0.8884955 (12610)	total: 1h 34m 59s	remaining: 17m 59s
12611:	learn: 0.8739264	test: 0.8884926	best: 0.8884955 (12610)	total: 1h 35m	remaining: 17m 59s
12612:	learn: 0.8739314	test: 0.8885134	best: 0.8885134 (12612)	total: 1h 35m	remaining: 17m 58s
12613:	learn: 0.8739363	test: 0.8885174	best: 0.8885174 (12613)	total: 1h 35m	remaining: 17m 58s
12614:	learn: 0.8739379	test: 0.8885327	best: 0.8885327 (12614)	total: 1h 35m 1s	remaining: 17m 57s
12615:	learn: 0.8739406	test: 0.8885501	best: 0.8885501 (12615)	total: 1h 35m 1s	remaining: 17m 57s
12616:	learn: 0.8739416	test: 0.8885536	best: 0.8885536 (12616)	total: 1h 35m 2s	remaining: 17m 56s
12878:	learn: 0.8747625	test: 0.8893654	best: 0.8893769 (12870)	total: 1h 37m 6s	remaining: 15m 59s
12879:	learn: 0.8747653	test: 0.8893709	best: 0.8893769 (12870)	total: 1h 37m 6s	remaining: 15m 59s
12880:	learn: 0.8747653	test: 0.8894228	best: 0.8894228 (12880)	total: 1h 37m 7s	remaining: 15m 58s
12881:	l

13218:	learn: 0.8757658	test: 0.8904054	best: 0.8904443 (13210)	total: 1h 39m 46s	remaining: 13m 26s
13219:	learn: 0.8757638	test: 0.8903950	best: 0.8904443 (13210)	total: 1h 39m 47s	remaining: 13m 26s
13220:	learn: 0.8757651	test: 0.8904154	best: 0.8904443 (13210)	total: 1h 39m 47s	remaining: 13m 25s
13221:	learn: 0.8757727	test: 0.8904164	best: 0.8904443 (13210)	total: 1h 39m 48s	remaining: 13m 25s
13222:	learn: 0.8757776	test: 0.8904103	best: 0.8904443 (13210)	total: 1h 39m 48s	remaining: 13m 24s
13223:	learn: 0.8757795	test: 0.8904141	best: 0.8904443 (13210)	total: 1h 39m 49s	remaining: 13m 24s
13224:	learn: 0.8757823	test: 0.8904238	best: 0.8904443 (13210)	total: 1h 39m 49s	remaining: 13m 23s
13225:	learn: 0.8757837	test: 0.8904119	best: 0.8904443 (13210)	total: 1h 39m 50s	remaining: 13m 23s
13226:	learn: 0.8757920	test: 0.8904296	best: 0.8904443 (13210)	total: 1h 39m 50s	remaining: 13m 23s
13227:	learn: 0.8757959	test: 0.8904281	best: 0.8904443 (13210)	total: 1h 39m 51s	remaining

13581:	learn: 0.8768536	test: 0.8915921	best: 0.8915947 (13578)	total: 1h 42m 37s	remaining: 10m 42s
13582:	learn: 0.8768531	test: 0.8915945	best: 0.8915947 (13578)	total: 1h 42m 37s	remaining: 10m 42s
13583:	learn: 0.8768496	test: 0.8915920	best: 0.8915947 (13578)	total: 1h 42m 38s	remaining: 10m 41s
13584:	learn: 0.8768541	test: 0.8915960	best: 0.8915960 (13584)	total: 1h 42m 38s	remaining: 10m 41s
13585:	learn: 0.8768565	test: 0.8916010	best: 0.8916010 (13585)	total: 1h 42m 38s	remaining: 10m 41s
13586:	learn: 0.8768590	test: 0.8916168	best: 0.8916168 (13586)	total: 1h 42m 39s	remaining: 10m 40s
13587:	learn: 0.8768662	test: 0.8916303	best: 0.8916303 (13587)	total: 1h 42m 39s	remaining: 10m 40s
13588:	learn: 0.8768714	test: 0.8916339	best: 0.8916339 (13588)	total: 1h 42m 40s	remaining: 10m 39s
13589:	learn: 0.8768712	test: 0.8916140	best: 0.8916339 (13588)	total: 1h 42m 40s	remaining: 10m 39s
13863:	learn: 0.8776368	test: 0.8924542	best: 0.8924542 (13863)	total: 1h 44m 47s	remaining

14219:	learn: 0.8786363	test: 0.8934844	best: 0.8935048 (14216)	total: 1h 47m 37s	remaining: 5m 54s
14220:	learn: 0.8786350	test: 0.8934873	best: 0.8935048 (14216)	total: 1h 47m 37s	remaining: 5m 53s
14221:	learn: 0.8786432	test: 0.8934723	best: 0.8935048 (14216)	total: 1h 47m 38s	remaining: 5m 53s
14222:	learn: 0.8786461	test: 0.8934882	best: 0.8935048 (14216)	total: 1h 47m 38s	remaining: 5m 52s
14223:	learn: 0.8786516	test: 0.8934947	best: 0.8935048 (14216)	total: 1h 47m 39s	remaining: 5m 52s
14224:	learn: 0.8786509	test: 0.8935100	best: 0.8935100 (14224)	total: 1h 47m 39s	remaining: 5m 51s
14225:	learn: 0.8786571	test: 0.8935139	best: 0.8935139 (14225)	total: 1h 47m 40s	remaining: 5m 51s
14226:	learn: 0.8786571	test: 0.8935137	best: 0.8935139 (14225)	total: 1h 47m 40s	remaining: 5m 51s
14227:	learn: 0.8786608	test: 0.8934937	best: 0.8935139 (14225)	total: 1h 47m 41s	remaining: 5m 50s
14228:	learn: 0.8786678	test: 0.8935253	best: 0.8935253 (14228)	total: 1h 47m 41s	remaining: 5m 50s


14558:	learn: 0.8796040	test: 0.8942642	best: 0.8942642 (14558)	total: 1h 50m 17s	remaining: 3m 20s
14559:	learn: 0.8796071	test: 0.8942712	best: 0.8942712 (14559)	total: 1h 50m 18s	remaining: 3m 19s
14560:	learn: 0.8796126	test: 0.8942636	best: 0.8942712 (14559)	total: 1h 50m 18s	remaining: 3m 19s
14561:	learn: 0.8796169	test: 0.8942525	best: 0.8942712 (14559)	total: 1h 50m 19s	remaining: 3m 19s
14562:	learn: 0.8796261	test: 0.8942601	best: 0.8942712 (14559)	total: 1h 50m 19s	remaining: 3m 18s
14563:	learn: 0.8796217	test: 0.8942791	best: 0.8942791 (14563)	total: 1h 50m 20s	remaining: 3m 18s
14564:	learn: 0.8796305	test: 0.8942724	best: 0.8942791 (14563)	total: 1h 50m 20s	remaining: 3m 17s
14565:	learn: 0.8796330	test: 0.8942800	best: 0.8942800 (14565)	total: 1h 50m 21s	remaining: 3m 17s
14566:	learn: 0.8796369	test: 0.8942907	best: 0.8942907 (14566)	total: 1h 50m 21s	remaining: 3m 16s
14830:	learn: 0.8803873	test: 0.8950468	best: 0.8950888 (14827)	total: 1h 52m 28s	remaining: 1m 16s


In [34]:
f1_score(val_predict.reshape(-1), np.array(val_offline['label']), average = 'weighted')

0.9158851368128152

In [35]:
sorted(dict(zip(model.feature_names_, model.feature_importances_)).items(), key = lambda x:x[1], reverse=True)

[('mf_pred', 7.788646989913974),
 ('p_std', 3.3468661810012676),
 ('date', 3.0410267567752425),
 ('target_avg_week_20', 1.5976241550184258),
 ('p_mean', 1.3198399151074776),
 ('topo_group', 1.052275773756264),
 ('current_feature_status_5', 0.9861901834033556),
 ('target_avg_lastweek_30', 0.9772702590213707),
 ('target_avg_beforelastday_30', 0.7327239186955011),
 ('target_avg_lastweek_20', 0.7296164576651568),
 ('target_avg_lastday_30', 0.7073596923583476),
 ('current_feature_speed_5', 0.6978901330097061),
 ('time_diff', 0.6778430899676909),
 ('target_avg_lastweek_5', 0.6595877137354743),
 ('link_avg_lastday', 0.646569714361603),
 ('time_avg_lastweek', 0.6339433116285369),
 ('P14', 0.6254552394722946),
 ('target_avg_lastweek_10', 0.5964800476310368),
 ('target_avg_lastday_5', 0.5889323217598716),
 ('P4', 0.5743767356025089),
 ('link_avg_lastweek', 0.5735007213656966),
 ('target_avg_lastday_10', 0.5693924221999443),
 ('target_avg_lastday_15', 0.5448005829370748),
 ('P22', 0.5410772113407

In [39]:
test_online['result'] = model.predict(test_online[[x for x in train_offline.columns if x not in non_features]], prediction_type='Class')

In [41]:
upload = test_online[['link','current_time','predict_time','result']]

upload.columns = ['link','current_slice_id','future_slice_id','label']

upload.to_csv('1206_wish_me_good_luck.csv',index=False)